# Summary

*Author: Benjamin Bradshaw*

A *huge* piece of whether or not an inferential or predictive project is successful is whether or not the correct information is included in the model. Often times when dealing with IoT data, the raw signal collected from a device is not appropriate for direct input into the model. For example, if we were interested in using step data on a given day to predict whether or not that day was recorded on a weekday or a weekend, including each minute's step count for that day would result in 1440 features (machine learnign verbiage for variables). Unless we had a large amount of data to learn from and fed the raw signal into a complex model such as a neural network (which is possible these days) that might not be th ebest approach. A more labor intensive approach in a data constrained environment might be to conduct what is known as *feature engineering*.

*Feature engineering* consists of taking raw data inputs and transforming them into such a way that extracts more signal, or captures specific characteristics of the data that are important for the task at hand. Feature engineering is not easy- mostly because it requires creativity on the part of *you* the practitioner to deeply understand the problem you are trying to solve, and then engineer features that allow a machine learning model to capture those characteristics. A large part of the preliminary detective work required for successful feature engineering comes in the form of *exploratory data analysis* which is just a fancy word for looking at the data and determining what aspects are important to capture in order to successfully extract the signal pertaining to your project.

This notebook will cover the following topics:
- Using exploratory data analysis (EDA) to identify useful aspects of the data that assist in our analysis task
- Engineer features that capture the patterns identified during EDA

In order motivate these tasks we construct the following prediction task: Can we utilize the accelerometer data in conjuction with other data available through NHANES to differentiate data collected on a weekday compare to data collected on a weekend?

# Learning objectives

By the time you work through this notebook you should be able to:
- Understand the need for splitting our data into a training and testing set when implementing a classification task, as well as how to correctly split our data depending on the use case at hand
- Use some of the basic libraries used in pythn for EDA including pandas, matplotlib, and seaborn
- Build basic software that allows efficient scaling of data analysis
- Understand how features may be extracted from raw accelerometer data for the purpose of a classification task

# Initialize

Dependencies. . . 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import utils # Utility functions containained in ./utils/
import scipy

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.set_option('max.rows', 100)
pd.set_option('max.columns', 100)

# Change this location to the path where you would like your data saved to
data_dir = '/Users/bbradshaw/nhanes/'

# Path to hdf store we will create later
hdf_path = 'nhanes.h5'

In [2]:
# Load cleaned accelerometer data from last time
%time pax = pd.read_hdf(os.path.join(data_dir, hdf_path), 'pax_clean')

CPU times: user 956 ms, sys: 958 ms, total: 1.91 s
Wall time: 2.26 s


In [3]:
# Remove 'pax' prefix on column names: it was getting annoying
pax.columns = [x.split('pax')[1] if 'pax' in x else x for x in pax.columns]

In [4]:
pax.head()

,seqn,stat,cal,day,n,hour,minut,inten,step,minute_of_day
0,31128.0,1,1,1,1,0,0,166,4.0,0
1,31128.0,1,1,1,2,0,1,27,0.0,1
2,31128.0,1,1,1,3,0,2,0,0.0,2
3,31128.0,1,1,1,4,0,3,276,4.0,3
4,31128.0,1,1,1,5,0,4,0,0.0,4


# The need for a training and testing set

Traditional statistics often times is concerned with creating *estimators* that are unbiased or perhaps consistent estimates of some true population parameter $\theta$. In this setting it is common to "fit" an estimator (e.g. linear/logistic regression etc) to the *entire* sample of data. In this setting this choice makes sense, since we seek to minimize the difference between $\theta$ and $\hat{\theta}$ (our estimated parameter). Certain estimators have guarantees about unbiasedness, and we maximize our statistical power when we use the entire sample to fit the model (think about what happens to the standard errors of the OLS coefficients as $n \rightarrow \infty$). However, at the end of the day an inferential analysis always suffers from doubt: apart from collecting a second (or third, or fourth) sample, we have no bullet proof way to check how robust our results are.

In a *prediction* setting our goal is quite different. Given a set of inputs $X$ we want our model to do a "good" job of minimizing the "difference" between the actual label associated with that input $y$ and the predicted label the model produces $\hat{y}$. Because this is our goal, we have a very powerful tool on our side to asses whether or not our model is "good". The tool is conceptually very simple and easy to understand: we will split our training set into two sets: a "training" set and a "testing" set. The training set will be used for all exploratory work, model training, model selection, hyperparameter tuning etc. The testing set will be set aside, completely quarantined from our development process, and only used once our model is entirely finished. At that point, we will use our model to compute predictions over the test set and we will use these predictions to assess just how good our model would have been "out in the wild" when making predictions on unseen examples.

Constructing a test set is conceptually simple, but it is fraught with danger in implementation. How you construct this split requires careful forethought about the goal of the task, as well as attention to the details that might imperil the generalizeability of your model.

In this section we will construct a train-test split of our dataset, walking through some of the common issues that you might want to consider when building a pipeline of your own.

## Creating a testing-training split

Let's get into the details of creating a train-test split. First we need to decide what the classification use case is. We know we are trying to classify weekdays and weekend days, but do we want our model to generalize to *new unseen people* or do we want our model to generalize to *new unseen days* on the same people. There is a subtle distinction between those two cases. In the first case, we would want to make sure people observed in our training set are not observed in our test set in order to ensure we simulate the ability to generalize to new people. In the second case it's actually OK if the same people show up in both the training and testing splits: we are training the model to learn individual specific patterns and behaviors.

For this analysis let's say our goal is option 1: generalize to new people. Since this is the case we need to ensure that days from people in our chosen training set do not also end up in our test set. We'll also want to ensure that the statistical properties of our training set match that of our test set as closely as possible. 

We'll start by taking a look at the demographic table found in NHANES. We'll then conduct the following steps:
- Filter the demographics table to only include participants who also exist in the pax table
- Conduct stratified sampling on age/gender/other important characteristics?

# Joining Data

In order to do some interesting EDA, we probably want to bring in more data types than just the raw accelerometer data.

The NHANES dataset has **A LOT** of tables. These tables are *relational* in structure meaning that their is a heirarchical structure that relates one set of data to another. In the NHANES dataset the *person level identifier* is the ```seqn``` number as you have already seen above. For some tables each row corresponds to exactly one ```seqn``` number. For other tables (like the raw activity data we have already worked with) there is a *one to many* mapping between ```seqn``` identifiers and row numbers, meaning that one ```seqn``` number corresponds to multiple input rows. Indeed for the ```pax``` dataset we already worked with above, each person is associated with *thousands* of rows corresponding to each minute of a day in which the person wore an activity monitor.

For this next section we will ensure that we can join together data from the various tables we pulled. This will be important for creating new interesting features later on, both for prediction and inference tasks, as we'll need to put together a feature matrix that combines data from multiple tables within the NHANES database.

## Example: Joining Datasets

As an example let's join together two of the tables we pulled in above that should now exist in your local HDF store:

- Participant demographics
- Physical activity data (we already pulled this in above)

There is a 1:1 correspondence between these two tables. For this join, we will do what is known as an *inner join*. This means that we will specify a join key that exists in both sets, and *only* join those keys that exist in the intersection of the two key sets. For more information on the different types of joins check out [this resource](https://www.w3schools.com/sql/sql_join.asp).

In [5]:
# Recall in the last notebook we read this from the NHANES website
demo_df = pd.read_hdf(os.path.join(data_dir, hdf_path), 'demographics_with_sample_weights')

# Peek inside
demo_df.head()

,seqn,sddsrvyr,ridstatr,ridexmon,riagendr,ridageyr,ridagemn,ridageex,ridreth1,dmqmilit,dmdborn,dmdcitzn,dmdyrsus,dmdeduc3,dmdeduc2,dmdschol,dmdmartl,dmdhhsiz,dmdfmsiz,indhhinc,indfminc,indfmpir,ridexprg,dmdhrgnd,dmdhrage,dmdhrbrn,dmdhredu,dmdhrmar,dmdhsedu,sialang,siaproxy,siaintrp,fialang,fiaproxy,fiaintrp,mialang,miaproxy,miaintrp,aialang,wtint2yr,wtmec2yr,sdmvpsu,sdmvstra
0,31127,4.0,2.0,2.0,1.0,5.397605e-79,11.0,12.0,3.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,4.0,0.75,NaN,2.0,21.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,6434.950248,6571.396373,2.0,44.0
1,31128,4.0,2.0,1.0,2.0,1.100000e+01,132.0,132.0,4.0,NaN,1.0,1.0,NaN,4.0,NaN,1.0,NaN,7.0,6.0,8.0,5.0,0.77,2.0,1.0,47.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,9081.700761,8987.041810,1.0,52.0
2,31129,4.0,2.0,2.0,1.0,1.500000e+01,189.0,190.0,4.0,NaN,1.0,1.0,NaN,10.0,NaN,1.0,5.0,6.0,6.0,10.0,10.0,2.71,NaN,1.0,41.0,1.0,4.0,1.0,4.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,5316.895215,5586.719481,1.0,51.0
3,31130,4.0,2.0,2.0,2.0,8.500000e+01,NaN,NaN,3.0,2.0,1.0,1.0,NaN,NaN,4.0,NaN,2.0,1.0,1.0,4.0,4.0,1.99,NaN,2.0,85.0,1.0,4.0,2.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,29960.839509,34030.994786,2.0,46.0
4,31131,4.0,2.0,2.0,2.0,4.400000e+01,535.0,536.0,4.0,2.0,1.0,1.0,NaN,NaN,4.0,NaN,1.0,4.0,4.0,11.0,11.0,4.65,2.0,1.0,36.0,1.0,5.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,26457.708180,26770.584605,1.0,48.0


What in the world? What are all the cryptic column names?? Well, the CDC has chosen somewhat unhelpful names that map to demographic characteristics of participants. We *could* go to the website that lists the variable names and keep track of them. However, if we are clever there might be a better way. . . .The pandas library makes it easy to scrape tables via a url. All we need is the url where the table is located, as well as the index of the xml blob we are interested in.

In [13]:
demo_metadata_url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics&CycleBeginYear=2005'
idx = 1 # I looked up the blob I was interested in in advance

# Create demographic metadata DataFrame
demo_metadata = pd.read_html(demo_metadata_url)[idx]

# Filter metadata to just include the demographics table
demo_metadata = demo_metadata[demo_metadata['Data File Name']=='DEMO_D']

# Peek inside
demo_metadata.head()

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints
0,AIALANG,Language of the MEC ACASI Interview Instrument,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
1,DMDBORN,In what country {were you/was SP} born?,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
2,DMDCITZN,{Are you/Is SP} a citizen of the United States...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
3,DMDEDUC2,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
4,DMDEDUC3,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None


Nice! Now we have a dataframe with two columns of interest: ```Variable Name``` and ```Variable Description```. We could remap the column names to human readable. I'll leave that as an exercise for you if you feel inclined. The metadata table tells us what the varible name codes refer to but unfortunately they don't tell us what the specific factor levels of each variable map to. In order to get that information we need to refer to the [NHANES documentation](https://wwwn.cdc.gov/nchs/nhanes/2005-2006/DEMO_D.htm#RIDEXMON).

Now that we have the demographic data let's join it to the ```pax``` dataset.

In [ ]:
# The demo seqn value is an int and the pax seqn is a string represented float
# thus the join will fail if we don't convert.
# TODO fix seqn in pax_raw before writing clean to fix this situation
demo_df['seqn'] = demo_df.seqn.astype(float).astype(str)

# Merge pax and demo into a single DataFrame
analysis_set = pax.merge(demo_df, on='seqn', how='inner')

In [ ]:
# Creating a new DataFrame with pax + demographics
analysis_set = pax.merge(demo_df, on='seqn', how='inner')